In [1]:
import itertools
import os
import subprocess
import multiprocessing
from tqdm import tqdm
import argparse
from datetime import datetime
import yaml

In [2]:
number_of_runs = 10
n_processes = 6

multiprocessing.cpu_count() 

4

In [3]:
def run_command(command):
    result = subprocess.run(command, capture_output=True)
    assert (
        result.returncode == 0
    ), f"""
    command:
    {"$ "+' '.join(command)}
    returncode: {result.returncode}
    stderr:
    {result.stderr.decode()}
    stdout:
    {result.stdout.decode()}
    """
    return result

def launcher(arg):
    instance_name, command, repetition = arg

    result = run_command(command)

    stdout_log = result.stdout.decode()

    return (instance_name, repetition, stdout_log)

In [4]:
def imap_unordered_bar(func, args, total, n_processes):
    p = multiprocessing.Pool(n_processes)

    with tqdm(total=total) as pbar:
        for i, result in tqdm(
            enumerate(p.imap_unordered(func, args)), desc="iner", disable=True
        ):
            instance_name, repetition, stdout_log = result
            pbar.update(1)
            if repetition == number_of_runs - 1:
                pbar.write(f"{instance_name} is completed at {datetime.now()}")
                pbar.write(stdout_log)
    pbar.close()
    p.close()
    p.join()


In [5]:
ttp_files = []
with open("TTP_benchmark/benchmarkfilelist.txt", "r") as f:
    for line in f:
        ttp_files.append(line[:-1])

ttp_files = ttp_files[:4]

### CS2SA*

In [6]:
os.chdir("CS2SA_star")

In [7]:
with open("output/CS2SA", "w") as f:
    pass

In [ ]:
number_of_runs = 10
n_processes = 6
timelimit = 600

args = []
for instance_name in ttp_files:
    for repetition in range(number_of_runs):
        command = [
            "java",
            "-jar",
            "TTPSolver.test.jar",
            instance_name,
            "cs2sa",
            "CS2SA",
            str(timelimit),
            instance_name[0:-4] + f"_{repetition}.sol"
        ]
        args.append((instance_name, command, repetition))

imap_unordered_bar(launcher, args, len(args), n_processes)